## REDSEA python version BPC For multiple images at a time

Further edits of the other steinbock integrated jupyter notebook in this folder. This script is made to ingest entire folder of data at once

Dataset I had been using for CD3 / CD20 compensation: https://zenodo.org/records/8023452

Passed through steinbock to derive .tiffs and masks, then fed into this script

The directory structure required for the script as-written is the one naturally produced by steinbock:

A master directory with two folders: \img & \masks - each containing .tiffs with the original images and the DeepCell generated masks, repectively, with matching file names - and 1 .csv file (panel.csv autog generated by steinbock)

WARNING! The script has not been thoroughly tested, but should produce outputs. I still trying to examine it carefully to be absolutely sure that it is doing what I intended!  WARNING! The script is currently producing an error where the cell-cell matrix loop is not identifying cell borders for some cells (unsure as to why)

In [136]:
# Package Imports
import PIL
from PIL import Image, ImageSequence, ImageOps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage 
import skimage.measure
import skimage.morphology
import glob
from scipy.io import loadmat
import time

import tifffile
import os

In [137]:
######## Edit these for your run!
# file locations
mainPath = 'C:\\Users\\caiello\\Desktop\\RedSEA practice\\practice' # main folder must contain \img and \masks folders and a panel.csv file (matching steinbock output)

# for multiple images at once:
file_list = []
for i in os.listdir(mainPath + "\\img"):
    i = i.replace(".tiff","")
    file_list.append(i)

# or set file_name manually
file_single = 'RNANeg_Tonsil_003'  #do not include the .tiff file type!

#  select which channel to normalize
normChannels = [13,18]    # I use numbers here, can switch to names as in your panel file -- if you set "channel_names_numbers = False" in the the file_reader call in the next cell 

# parameters for compensation (change as desired)
REDSEAChecker = 1 # 1 means subtract+ reinforce
elementShape = 2 # star, 1 == square size
elementSize = 2 # star or square extension size


# output path
pathResults = mainPath + '/intensities' # output location, set as intensities here to match steinbock output
try:
    os.listdir(pathResults)
except:
    os.mkdir(pathResults)

alt_path = "\\an\\alternate\folder\\in\\which\\to\\put\\your\\non-scaled\\and\\non-compensated\\data" ## use if you want the non-compensated / non-scaled data, but don't want them crowding the main steinbock intensities folder
######### if you use alt_path, manually make the directory in file explorer!!!

In [140]:
## Functions (normally, minimize this cell)
# helper function 1
def ismember(a, b):
    bind = {}
    for i, elt in enumerate(b):
        if elt not in bind:
            bind[elt] = i
    return [bind.get(itm, None) for itm in a]  # None can be replaced by any other "not in b" value

# helper function 2

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()
        
def file_reader(mainPath,file_name,channel_names_numbers = True):
    massDS_path = mainPath + '\\panel.csv' # csv file location, need
    pathTiff = mainPath + '\\img\\' + file_name + '.tiff' #  tiff location, links to a single .tiff
    pathMat = mainPath + '\\masks\\' + file_name + '.tiff' # corresponding .tiff's mask
        
    ####### Read in of files:
    ### read in panel.csv
    massDS = pd.read_csv(massDS_path) # read the mass csv
    if channel_names_numbers == True:
        clusterChannels = massDS[massDS['keep'] == 1].reset_index().index.values # only get the label column minus 1 is for zero-indexing (matched channel numbers with Napari)
                                                # remove -1 if you want 1-indexing of the channel names
    else:
        clusterChannels = massDS['name']  # if you want your channel names to be want is used
    channelNum = len(clusterChannels) # how many channels
    #print(massDS.head())
    
    #### this part reads in multichannel tiff file
    # read in the image and transform into a 'countsNoNoise' matrix
    array_list=[]
    for channel in clusterChannels:
        t=tifffile.imread(pathTiff)[channel]
        array_list.append(t)
    countsNoNoise=np.stack(array_list,axis=2) # count matrices in the image
    
    ###### Read in segmentation .tiff
        # Define the boundary region
        #### these code is just entire translation of redsea matlab v1.0
    Segmentation = tifffile.imread(pathMat).astype('int')
    cellNum = np.max(Segmentation) # how many labels
    stats = skimage.measure.regionprops(Segmentation+1) # get the regional props for all the labels

    ### make empty container matrices
    data = np.zeros((cellNum + 1,channelNum))
    dataScaleSize = np.zeros((cellNum + 1,channelNum))
    cellSizes = np.zeros((cellNum + 1,1))
    
    # this part extract counts data from the whole cell regions, for each individual cells etc
    
    for i in range(cellNum + 1): # for each cell (label)
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i,0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes

    return clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize

def cell_cell_matrix(Segmentation, cellNum, REDSEAChecker = REDSEAChecker):
    # this block is for computing cell cell matrix
    [rowNum, colNum] = Segmentation.shape
    cellPairMap = np.zeros(((cellNum + 1),(cellNum + 1))) # cell-cell shared perimeter matrix container
    
    # start looping the mask and produce the cell-cell contact matrix
    for i in range(rowNum):
        if i == 0:
            a = 0
            c = 2
        elif i == 999:
            a = 1
            c = 1
        else:
            a = 1
            c = 2
        for j in range(colNum):
            if j == 0:
                b = 0
                d = 2
            elif j == 999:
                b = 1
                d = 1
            else:
                b = 1
                d = 2
            tempMatrix = Segmentation[i-a:i+c,j-b:j+d] # the 3x3 window, centered on the point i,j
            #print(tempMatrix)
            tempFactors = np.unique(tempMatrix).astype('int') #unique
            #print(tempFactors)
            centerpoint_value = Segmentation[i,j]
            #print(centerpoint_value)
            for k in tempFactors:
                if k != centerpoint_value: # only add to the cellPairMap for the centerpoint pixel -- this prevents multiplicate counting
                    #print("trigger")
                    cellPairMap[centerpoint_value,k] = cellPairMap[centerpoint_value,k] + 1  
        
    # converting the cell cell maps to fraction of cell - cell boundary (not of total cell boundary [!?] -- as in boundary with empty space not counted)
    cellPairNorm = np.zeros(((cellNum+1),(cellNum+1)))
    for i in np.arange(0,len(cellPairMap)):
        if np.sum(cellPairMap[i]) > 0:
            cellPairNorm[i] = - (cellPairMap[i] / np.sum(cellPairMap[i]))
    cellPairNorm = cellPairNorm[1:,1:]
    cellPairNorm = cellPairNorm + REDSEAChecker*np.identity(cellNum ) 
    #display(pd.DataFrame(cellPairMap))
    #print(pd.DataFrame(cellPairMap).sum())
    return cellPairNorm, rowNum, colNum

def Cell_Edge_Intensities(Segmentation, cellNum, channelNum, countsNoNoise, rowNum, colNum, elementShape = 2, elementSize = 2):
    # now starts the calculation of signals from pixels along the boudnary of cells
    MIBIdataNearEdge1 = np.zeros((cellNum+1,channelNum))
    
    # start the boundary region selection and count extraction
    
    ##### A List of Items
    items = list(range(cellNum))
    l = len(items)
    printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50) # progress bar
    #####
    
    ######pre-calculated shape
    if elementShape==1: # square
        square=skimage.morphology.square(2*elementSize+1)
        square_loc=np.where(square==1)
    elif elementShape==2: # diamond
        diam=skimage.morphology.diamond(elementSize) # create diamond shapte based on elementSize
        diam_loc=np.where(diam==1)
    else:
        print("Error elementShape Value not recognized.")
    ############
    
    for i in range(cellNum):
        [tempRow,tempCol] = np.where(Segmentation==i)
        # sequence in row not col, should not affect the code
        for j in range(len(tempRow)):
            label_in_shape=[] # empty list in case
            # make sure not expand outside
            if (elementSize-1<tempRow[j]) and (tempRow[j]<rowNum-elementSize-2) and (elementSize-1<tempCol[j]) and (tempCol[j]<colNum-elementSize-2):
                ini_point = [tempRow[j]-elementSize,tempCol[j]-elementSize] # corrected top-left point
            
                if elementShape==1: # square
                    square_loc_ini_x=[item + ini_point[0] for item in square_loc[0]]
                    square_loc_ini_y=[item + ini_point[1] for item in square_loc[1]]
                    
                    label_in_shape=[Segmentation[square_loc_ini_x[k],square_loc_ini_y[k]] for k in range(len(square_loc_ini_x))]
                    
                elif elementShape==2: # diamond
                    diam_loc_ini_x=[item + ini_point[0] for item in diam_loc[0]]
                    diam_loc_ini_y=[item + ini_point[1] for item in diam_loc[1]]
                    # finish add to ini point
                
                    label_in_shape=[Segmentation[diam_loc_ini_x[k],diam_loc_ini_y[k]] for k in range(len(diam_loc_ini_x))]
            is_border_px = len(np.unique(label_in_shape))
            
            if is_border_px > 1:
                MIBIdataNearEdge1[i,:] = MIBIdataNearEdge1[i,:] + countsNoNoise[tempRow[j],tempCol[j],:]
        
        # Update Progress Bar
        printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
    return MIBIdataNearEdge1

def CalculateREDSEA(MIBIdataNearEdge1, cellPairNorm, data, channelNormIdentity, cellNum, dataScaleSize, cellSizes):    
    MIBIdataNorm2 = np.transpose(np.dot(np.transpose(MIBIdataNearEdge1[1:,:]),cellPairNorm))
    #this is boundary signal subtracted by cell neighbor boundary
    MIBIdataNorm2 = MIBIdataNorm2 + data[1:,:] # reinforce onto the whole cell signal (original signal)
    
    MIBIdataNorm2[MIBIdataNorm2<0] = 0 # clear out the negative ones
    # flip the channelNormIdentity for calculation
    rev_channelNormIdentity=np.ones_like(channelNormIdentity)-channelNormIdentity
    # composite the normalized channels with non-normalized channels
    # MIBIdataNorm2 is the matrix to return
    MIBIdataNorm2 = data[1:,:] * np.transpose(np.tile(rev_channelNormIdentity,(1,cellNum))) + MIBIdataNorm2 * np.transpose(np.tile(channelNormIdentity,(1,cellNum)))
    
    # the function should return 4 variables
    dataCells = data[1:,:]
    dataScaleSizeCells = dataScaleSize[1:,:]
    dataCompenCells = MIBIdataNorm2
    dataCompenScaleSizeCells = MIBIdataNorm2 / cellSizes[1:,:]

    return dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells

def data_matrix_with_label_and_cellsizes(data_in, cellNum, cellSizes, clusterChannels):
    ### in a separate funcito because this data is not likely to be wanted. still it can be accessed:
    labelVec = [i for i in np.arange(1,cellNum + 1,1)]
    cellSizesVec_flat = [item for sublist in cellSizes[1:,:] for item in sublist] # flat the list
    dataL = pd.DataFrame({'Object':labelVec, 'cell_size':cellSizesVec_flat})
    dataCells_df=pd.DataFrame(data_in)
    dataCells_df.columns = clusterChannels
    data_out = pd.concat((dataL,dataCells_df),axis=1)
    return data_out

def Outputter(data_in, file_name, pathResults = pathResults):
    data_in.to_csv(pathResults + '\\' + file_name + ".csv", index = False)

def RunRedSEA(mainPath, file, channel_names_numbers = True, pathResults = pathResults, alt_path = pathResults, normChannels = normChannels, print_options = "only_compensated_and_scaled"):
    if print_options not in ['only_compensated_and_scaled','only_scaled','all']:
        raise("Error: print_options not one of the following: 'only_compensated_and_scaled', 'only_scaled', or 'all'!") 
    if type(file) == list:
        for ii in file:
            file_name = ii
            print("Starting RedSEA compensation of file: " + file_name)
            clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize = file_reader(mainPath, ii, channel_names_numbers = channel_names_numbers)
            normChannelsInds = ismember(normChannels,clusterChannels)
            channelNormIdentity = np.zeros((len(clusterChannels),1))
            # make a flag for compensation
            for i in range(len(normChannelsInds)):
                    channelNormIdentity[normChannelsInds[i]] = 1 
            # print("cellNum for " + file_name + " " + str(cellNum))
            cellPairNorm, rowNum, colNum = cell_cell_matrix(Segmentation = Segmentation, REDSEAChecker = REDSEAChecker, cellNum = cellNum)
            MIBIdataNearEdge1 = Cell_Edge_Intensities(Segmentation, countsNoNoise = countsNoNoise, cellNum = cellNum, channelNum = channelNum, rowNum = rowNum, colNum = colNum, elementShape = 2, elementSize = 2)
            dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells = CalculateREDSEA(MIBIdataNearEdge1 = MIBIdataNearEdge1, cellPairNorm = cellPairNorm, data = data, channelNormIdentity = channelNormIdentity, cellNum = cellNum, dataScaleSize = dataScaleSize, cellSizes = cellSizes)
            dataL_full = data_matrix_with_label_and_cellsizes(dataCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            dataScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            dataCompenL_full = data_matrix_with_label_and_cellsizes(dataCompenCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            dataCompenScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataCompenScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
            if print_options == "only_compensated_and_scaled":
                Outputter(dataCompenScaleSizeL_full, file_name = file_name)
            elif print_options == "only_scaled":
                Outputter(dataCompenScaleSizeL_full, file_name = file_name)
                Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA.csv", pathResults = alt_path)
            elif print_options == "all":
                Outputter(dataL_full, file_name = file_name + "_pre_REDSEA_unscaled.csv")
                Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA_scaled.csv")
                Outputter(dataCompenL_full, file_name = file_name + "_post_REDSEA_unscaled.csv")
                Outputter(dataCompenScaleSizeL_full, file_name = file_name)
    elif type(file) == str:
        file_name = file
        print("Starting RedSEA compensation of file: " + file_name)
        clusterChannels, countsNoNoise, Segmentation, cellNum, channelNum, data, cellSizes, dataScaleSize = file_reader(mainPath, file_name, channel_names_numbers = channel_names_numbers)
        normChannelsInds = ismember(normChannels,clusterChannels)
        channelNormIdentity = np.zeros((len(clusterChannels),1))
        # make a flag for compensation
        for i in range(len(normChannelsInds)):
                channelNormIdentity[normChannelsInds[i]] = 1 
        # print("cellNum for " + file_name + " " + str(cellNum))
        cellPairNorm, rowNum, colNum = cell_cell_matrix(Segmentation = Segmentation, REDSEAChecker = REDSEAChecker, cellNum = cellNum)
        MIBIdataNearEdge1 = Cell_Edge_Intensities(Segmentation, countsNoNoise = countsNoNoise, cellNum = cellNum, channelNum = channelNum, rowNum = rowNum, colNum = colNum, elementShape = 2, elementSize = 2)
        dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells = CalculateREDSEA(MIBIdataNearEdge1 = MIBIdataNearEdge1, cellPairNorm = cellPairNorm, data = data, channelNormIdentity = channelNormIdentity, cellNum = cellNum, dataScaleSize = dataScaleSize, cellSizes = cellSizes)
        dataL_full = data_matrix_with_label_and_cellsizes(dataCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        dataScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        dataCompenL_full = data_matrix_with_label_and_cellsizes(dataCompenCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        dataCompenScaleSizeL_full = data_matrix_with_label_and_cellsizes(dataCompenScaleSizeCells, cellNum = cellNum, cellSizes = cellSizes, clusterChannels = clusterChannels)
        if print_options == "only_compensated_and_scaled":
            Outputter(dataCompenScaleSizeL_full, file_name = file_name)
        elif print_options == "only_scaled":
            Outputter(dataCompenScaleSizeL_full, file_name = file_name)
            Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA.csv", pathResults = alt_path)
        elif print_options == "all":
            Outputter(dataL_full, file_name = file_name + "_pre_REDSEA_unscaled.csv")
            Outputter(dataScaleSizeL_full, file_name = file_name + "_pre_REDSEA_scaled.csv")
            Outputter(dataCompenL_full, file_name = file_name + "_post_REDSEA_unscaled.csv")
            Outputter(dataCompenScaleSizeL_full, file_name = file_name)
    return dataL_full, dataScaleSizeL_full, dataCompenL_full, dataCompenScaleSizeL_full # these are the files of only the last file, if running multiple files at once


In [141]:
# Whole folder run (this one had 4 files in it):
dataL_full, dataScaleSizeL_full, dataCompenL_full, dataCompenScaleSizeL_full = RunRedSEA(mainPath = mainPath, file = file_list, channel_names_numbers = True, normChannels = normChannels, print_options = "only_scaled")

Starting RedSEA compensation of file: RNANeg_Tonsil_001
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Starting RedSEA compensation of file: RNANeg_Tonsil_002
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Starting RedSEA compensation of file: RNANeg_Tonsil_003
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Starting RedSEA compensation of file: RNANeg_Tonsil_004
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [148]:
# pre-comp data from the last file in the run, scaled by cell size
dataScaleSizeL_full.head()

,Object,cell_size,0,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,24,25
0,1,12.0,0.083333,5.647098,3.013256,1.318903,1.036586,1.262176,0.000000,2.101948,...,0.351574,2.039562,4.189841,0.351362,0.385892,12.605571,18.170195,4.810619,27.770319,1.949942
1,2,12.0,0.000000,1.467000,0.922300,1.235178,0.218416,0.416667,0.083333,1.218475,...,0.000000,1.711730,0.928775,0.387283,0.252729,6.553608,9.611255,1.744451,28.690329,1.282992
2,3,11.0,0.090909,2.074444,1.400868,0.970991,0.214524,1.288590,0.344656,0.448292,...,0.207362,2.548671,1.080995,0.000000,0.230224,5.562967,7.117647,1.381895,22.161097,1.549177
3,4,8.0,0.000000,0.125000,1.232233,0.250000,0.644276,0.413830,0.000000,0.558378,...,0.519149,1.976839,0.455801,0.250000,0.250000,3.502497,5.479109,1.653704,40.382545,3.176192
4,5,15.0,0.000000,2.105961,1.196886,0.993774,0.323342,1.038583,0.144026,1.982418,...,0.396339,2.125142,1.058890,0.243160,0.066667,7.997893,12.763613,1.115945,8.551426,1.737276


In [149]:
# post-comp data from the last file in the run, scaled by cell size
dataCompenScaleSizeL_full.head()

,Object,cell_size,0,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,24,25
0,1,12.0,0.083333,5.647098,3.013256,1.318903,1.036586,1.262176,0.000000,2.101949,...,0.351574,2.039562,0.0,0.351362,0.385892,12.605570,18.170195,4.810619,27.770320,1.949942
1,2,12.0,0.000000,1.467000,0.922300,1.235178,0.218416,0.416667,0.083333,1.218475,...,0.000000,1.711730,0.0,0.387283,0.252729,6.553608,9.611255,1.744451,28.690328,1.282992
2,3,11.0,0.090909,2.074444,1.400868,0.970991,0.214524,1.288590,0.344656,0.448292,...,0.207362,2.548671,0.0,0.000000,0.230224,5.562967,7.117647,1.381895,22.161097,1.549177
3,4,8.0,0.000000,0.125000,1.232233,0.250000,0.644276,0.413830,0.000000,0.558378,...,0.519149,1.976839,0.0,0.250000,0.250000,3.502497,5.479109,1.653704,40.382545,3.176192
4,5,15.0,0.000000,2.105961,1.196886,0.993774,0.323342,1.038583,0.144026,1.982418,...,0.396339,2.125142,0.0,0.243160,0.066667,7.997893,12.763613,1.115945,8.551426,1.737276


In [47]:
# single file run:

dataL_full, dataScaleSizeL_full, dataCompenL_full, dataCompenScaleSizeL_full = RunRedSEA(mainPath = mainPath, file = file_single, channel_names_numbers = True, normChannels = normChannels, print_options = "only_compensated_and_scaled")

Starting RedSEA compensation of file: RNANeg_Tonsil_003
Progress: |██████████████████████████████████████████████████| 100.0% Complete
